# Univariate TDE-HMM

In this notebook, we apply a TDE-HMM on the time course of a single independent component (IC). The notebook is separated in three distinct parts:
- In the first part, we import or define the libraries and methods that will be usefull to execute the code. We also define the location of the where to find and store our data.
- In the second part, we specify the metaparameters and options we want for the execution of the code.
- In the third part, we run the TDE-HMM on our raw data with specified parameters, then store it in netCDF files. Finally, we can decide to plot some interesting results on figures.

## Libraries, Functions, Storage

##### Libraries and functions

In [1]:
# Storage management
import pickle   # The inferred model parameters are stored in a .pkl file
import h5py   # Manages .mat files in Python
from loader import load_oneIC  # Stores temporarily useful data from our .mat data file in a dictionary
import xarray as xr   # Manages .nc (netCDF) files in Python.
                      # The states' informations are stored in a .nc file for each subject.

# Scientific computing
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from hmmlearn import hmm
import scipy.signal as signal
import mne
from wavelet_transform import wavelet_transform

# Else
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import time as tm

##### Storage

`FCK_LOCKED_IC_JYOTIKA_250819.mat` contains 23x4 Matlab structures containing the data for each of the 23 subjects
and each IC when they exist. The existing ICs can be visualised in the `data_structure.png` screen capture, or using 
Matlab. Each of these structures, when they exist, contain the fields we can visualise in the `data_labels.png`
screen capture or using Matlab.

The way we access these data with Python can be understood by reading the loader.py script.

In [4]:
directory = "E:/timot/Documents/1 - Centrale Marseille/0.5 - Semestre S8/Stage/NIC_250819"
file = "FCK_LOCKED_IC_JYOTIKA_250819.mat"

path = f"{directory}/{file}"

mat_file = h5py.File(path, "r")
cells_refs = mat_file['FCK_LOCKED_IC_JYOTIKA']

n_IC = 4
n_subj = 23

`data_dir` and `figures_dir` are the directories in which you will store and search for the data and the figures the algorithm produces.

In [5]:
figures_dir = "tde-hmm/Univariate/Run subj by subj - 10 lags and no PCA/figures/"
data_dir = "tde-hmm/Univariate/Run subj by subj - 10 lags and no PCA/data/"

##### Other useful functions

The embedx function copies the `x` array len(lags) times into `xe` with lags (i.e. time delays) between `lags[0]` and `lags[-1]` (we implement the time-delay array for the HMM).

In [6]:
def embedx(x, lags):
    
    Xe = np.zeros((x.shape[1], x.shape[0],  len(lags)))

    for l in range(len(lags)):
        Xe[:, :, l] = np.roll(x, lags[l], axis=0).swapaxes(0, 1)

    # Remove edges
    valid = np.ones((x.shape[0], 1), dtype=np.int8)
    valid[:np.abs(np.min(lags)), :] = 0
    valid[-np.abs(np.max(lags)):, :] = 0

    Xe = Xe[:, valid[:, 0] == 1, :]

    return Xe, valid

The `statesPSD` function returns the Power Spectral Density (PSD) characterising each state of the inferred HMM using the Welch method on the bits of raw time-courses in which the state is 'on' (probability of presence > 2/3). It is a non-parametric method since we don't use the inferred parameters of the HMM to compute this PSD. The frequency axis corresponding with the computed PSD is also returned with `freqs`.

In [7]:
def statesPSD(gamma, n_states, xe, fs=256/3):

    psd_all = []
    for i in range(n_states):

        # Compute PSD separately for each lag
        tot = []
        for seg in xe[gamma[:, i]>(2/3), :].T:
            freqs, psd = signal.welch(x=seg, fs=fs, nfft=1000)
            tot.append(psd)
        psd = np.mean(np.asarray(tot), 0)
        psd_all.append(psd)
    
    psd_all = np.asarray(psd_all)
    
    return freqs, psd_all

## Parameters

In [8]:
### Imput data parameters
# subj_list = [2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 14, 16, 17, 18, 19, 20, 21, 22, 23] # All except 1, 9, 13, 15
# subj_list = [2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 14, 16, 17, 18, 19, 22, 23] # same && IC1 exists
subj_list = [2]
# IC_list = [1, 2, 3, 4]
IC_list = [1, 2, 3]
downsamp_rate = 3  # The downsampling rate of the raw time-courses
# downsamp_rate = 1
## start_idx and end_idx are the indexes of the trial time-courses on which you want to start and end the HMM.
start_idx = 0  # Corresponds to the beginning of the trials, i.e. -4s.
# start_idx = 4*256  # Corresponds to the time-locking (i.e.0s)
end_idx = -1  # Corresponds to the end of the trial (i.e. 3s)
# end_idx = 4*256  # Corresponds to the time-locking (i.e.0s)
lags = np.arange(-5, 5)  # The time-delay embedding
# lags = np.arange(-29, 29)
# lags = np.arange(-11, 12)
n_lags = lags.shape[0]
apply_PCA = False  # Do we apply a PCA before inferring the HMM?
n_components = 0     # Number of principal components in case of PCA
# n_components = 40

### HMM parameters
all_subj_first = False   # Do we compute the model on all subjects before refining it for each subject?
covariance_type = 'full'
# covariance_type = 'diag'  # ONLY IN CASE OF PCA
n_iter = 1000000  # The maximum number of iterations of the EM algorithm if we don't meet the next requirement.
tol = 0.01  # The EM algorithm will stop if the maximum likelihood of the HMM 
            # varies under this tolerence for each iteration.

### Output data parameters
# n_states_list = [3, 4, 5, 6]    # Number of hidden Markov states. Must be a list.
n_states_list = [3]

## Script

##### Inferring the HMM on all subjects of `subj_list` concatenated together

In [9]:
if all_subj_first:
    for IC in IC_list:
        print(f"------ IC{IC}: Model inferred on ALL SUBJECTS ------")
        print("Computing the imput matrix for the model")
        subj_lengths = []
        tde_imput = []
        for subj in subj_list:
            
            # Loading all data for subject{subj} IC{IC}
            data, n_trials = load_oneIC(mat_file, cells_refs, subj, IC)
            big_timecourse = np.concatenate([data[f'raw_timecourse_256Hz'][i][::downsamp_rate] for i in range(100)])
                # Downsampled to 256/3 = 85,33333Hz
            big_timecourse = scale(big_timecourse)
            x = big_timecourse.reshape(-1, 1)
            xe, valid = embedx(x, lags)
            
            if apply_PCA:
                pca = PCA(n_components=n_components)
                y = pca.fit_transform(xe[0, :, :])
#             print(f"IC{IC} loaded")
            else:
                y = xe[0, :, :]
            tde_imput.append(y)
            subj_lengths.append(xe.shape[1])
        tde_imput = np.concatenate(tde_imput)

        for n_states in n_states_list:
            start_time = tm.time()
            print(f"Computing and saving the model for {n_states} states")
            model = hmm.GaussianHMM(n_components=n_states, n_iter=n_iter,
                                    covariance_type=covariance_type, tol=tol)
            model.fit(tde_imput, subj_lengths)

            with open(data_dir + f"ALLSUBJECTS-IC{IC}_st{n_states}_lg{n_lags}co{n_components}"
                + "UnivariateGaussianHMM.pkl", "wb") as file: pickle.dump(model, file)
            print(f"IC{IC}, {n_states} states: {int(tm.time() - start_time)} seconds")

##### Inferring the HMM on each subject of `subj_list` one after the other

In [14]:
complete_time = tm.time()
for n_states in n_states_list:
    print(f"****** {n_states} STATES ******")
    for subj in subj_list:
        print(f"    ** SUBJECT{subj} **    ")
        tcourses = []
        psds = []
        IC_new_list = []
        for IC in IC_list:
            start_time = tm.time()
            try:
                print(f"---- SUBJECT{subj}, IC{IC} - {n_states} states ----")
                if all_subj_first:
                    print("Loading initialisation parameters")
                    file = open(data_dir + f"ALLSUBJECTS-IC{IC}_st{n_states}_lg{n_lags}co{n_components}"
                            + "UnivariateGaussianHMM.pkl", "rb")
                    model = pickle.load(file)
                    model.init_params = 'st'
                # Loading all data for subject{subj}, IC{IC}
                data, n_trials = load_oneIC(mat_file, cells_refs, subj, IC)
                print("Computing and saving the model")
                big_timecourse = np.concatenate([data[f'raw_timecourse_256Hz'][i][start_idx:end_idx:downsamp_rate] 
                                                 for i in range(n_trials)])
                big_timecourse = scale(big_timecourse)
                x = big_timecourse.reshape(-1, 1)
                xe, valid = embedx(x, lags)
                if apply_PCA:
                    pca = PCA(n_components=n_components)
                    y = pca.fit_transform(xe[0, :, :])
                else:
                    y = xe[0, :, :]
                if all_subj_first==False:
                    model = hmm.GaussianHMM(n_components=n_states, n_iter=n_iter,
                                        covariance_type=covariance_type, tol=tol)
                model.fit(y)
                with open(data_dir + f"su{subj}IC{IC}_st{n_states}_lg{n_lags}co{n_components}"
                    + "UnivariateGaussianHMM.pkl", "wb") as file: pickle.dump(model, file)
                print("Computing the Probability time-course and the Power Spectral Density of each state")
                gamma = model.predict_proba(y)
                tcourse = np.concatenate(
                    (np.zeros((abs(lags[0]),n_states)), gamma, np.zeros((lags[-1],n_states)))
                )
                t_len = data["time_axis"][start_idx:end_idx:downsamp_rate].shape[0]
                tcourse_trials = np.zeros((n_trials, t_len, n_states))
                for tr in range(n_trials):
                    tcourse_trials[tr] = tcourse[tr*t_len:(tr+1)*t_len]
                tcourses.append(tcourse_trials[np.newaxis])
                freqs, psd = statesPSD(gamma, n_states, xe[0], fs=256/downsamp_rate)
                psd = psd[np.newaxis,]
                psds.append(psd)
                IC_new_list.append(IC)
                print(f"Model for subj{subj}, IC{IC} computed: {int(tm.time() - start_time)} seconds")
            except:
                print(f"The model for subj{subj}, IC{IC} was NOT computed")
        print(f"Saving the data for subj{subj}: Proba. time-course, Frac. occupancy and PSD")
        if len(tcourses)>1:
            tcourses = np.concatenate((tcourses))
            psds = np.concatenate((psds))
        else:
            tcourses = tcourses[0]
            psds = psds[0]
        ds = xr.Dataset(
            {
                "states_timecourse": (("IC", "trials","time", "states"), tcourses),
                "states_psd": (("IC", "states", "freq"), psds),
            },
            {
                "IC":IC_new_list,
                "time":data["time_axis"][start_idx:end_idx:downsamp_rate],
                "states":np.arange(1, n_states+1),
                "freq": freqs,
            }
        )
        ds = ds.assign(frac_occ = (ds["states_timecourse"].sum("trials")/ds.sizes["trials"]))
        ds.to_netcdf(data_dir + f"su{subj}_st{n_states}_lg{n_lags}co{n_components}_data.nc")


sec_time = tm.time() - complete_time
hr_time = int(sec_time/3600)
sec_time = sec_time - (hr_time*3600)
mn_time = int(sec_time/60)
sec_time = sec_time - (mn_time*60)
print(f"Run time: {hr_time}h{mn_time}mn{sec_time}s")

****** 3 STATES ******
    ** SUBJECT2 **    
---- SUBJECT2, IC1 - 3 states ----
Loading the raw timecourse
Computing and saving the model
Computing the Probability time-course and the Power Spectral Density of each state
Model for subj2, IC1 computed: 86 seconds
---- SUBJECT2, IC2 - 3 states ----
Loading the raw timecourse
Computing and saving the model
Computing the Probability time-course and the Power Spectral Density of each state
Model for subj2, IC2 computed: 87 seconds
---- SUBJECT2, IC3 - 3 states ----
Loading the raw timecourse
Computing and saving the model
Computing the Probability time-course and the Power Spectral Density of each state
Model for subj2, IC3 computed: 63 seconds
Saving the data for subj2: Proba. time-course, Frac. occupancy and PSD
Run time: 0h3mn56.973817586898804s


##### Plotting the fractional occupancies of the states, the averaged wavelet-transform and rhe states PSD on one big figure for each subject, each IC

In [ ]:
info = mne.create_info(ch_names=['signal'], sfreq=256, ch_types=['eeg'])

for n_states in n_states_list:
    widths = [14, 1, 5]
    heights = [1 for i in range(n_states)] + [4]
    gs_kw = dict(width_ratios=widths, height_ratios=heights, wspace=0.0, hspace=0.0)
    for subj in subj_list:
        ds = xr.open_dataset(data_dir + f"su{subj}_st{n_states}_lg{n_lags}co{n_components}_data.nc")
        start_t = ds["time"].values[0]
        end_t = ds["time"].values[-1]
        for IC in IC_list:
            idx = np.searchsorted(ds["IC"].values, IC)
            try:
                fig, f_axes = plt.subplots(figsize=(sum(widths),sum(heights)), ncols=3, 
                                            nrows=(n_states+1), constrained_layout=True,
                                            gridspec_kw=gs_kw)
                for state in ds["states"].values:
                    ax = f_axes[state-1,0]
                    ax.plot(ds["time"].values, ds["frac_occ"].values[idx, :, state-1], color=f"C{state-1}")
                    ax.set_xlim([-4, 3])
                    ax.set_ylabel(f"state {state}")

                ax = f_axes[n_states,0]
                data, n_trials = load_oneIC(mat_file, cells_refs, subj, IC)
                tfr = wavelet_transform(data, info, trial=np.arange(1,n_trials+1))
                mappable = ax.imshow(tfr[0], aspect='auto', origin='lower', extent=[-4, 3, 2, 50], 
                                       norm=colors.PowerNorm(gamma=0.5), cmap='RdYlBu_r')
                ax.set_ylabel(f'Frequencies IC{IC} (Hz)')
                ax.set_xlabel(f'Time (s)')
                fig.colorbar(mappable, cax=f_axes[n_states,1])

                ax = f_axes[n_states,2]
                ax.plot(ds["freq"].values, ds["states_psd"].values[idx,].T)
                ax.set_ylabel(f'PSD IC{IC}')
                ax.set_xlabel(f'Frequency (Hz)') 
                lines = ax.get_lines()
                labels = [f"State {i}" for i in range (1, n_states+1)]
                ax = f_axes[0,2]
                ax.legend(lines, labels, loc='upper left')

                fig.savefig(figures_dir + f'grid-{n_states}states-subj{subj}-IC{IC}.png', dpi=300)
                plt.close(fig)
            except:
                print(f"IC{IC} of subject {subj} does not exist")
                plt.close(fig)

##### Plotting the time-course of each states for `n_trials_to_plot` trials on a figure for each subject, each IC

In [ ]:
n_trials_to_plot = 5

for n_states in n_states_list:
    for subj in subj_list:
        fig, f_axes = plt.subplots(figsize=(20,n_states*n_trials_to_plot), ncols=1, 
                                   nrows=(n_states*n_trials_to_plot), constrained_layout=True)
        for IC in IC_list:
            ds = xr.open_dataset(data_dir + f"su{subj}_st{n_states}_lg{n_lags}co{n_components}_data.nc")
            try:
                idx = np.searchsorted(ds["IC"].values, IC)
                if ds["IC"].values[idx]==IC: 
                    for state in ds["states"].values:
                        color = f"C{state-1}"
                        for trial in range(1, n_trials_to_plot+1):
                            ax = f_axes[n_trials_to_plot*(state-1)+trial-1]
                            ax.fill_between(ds["time"], ds["states_timecourse"].values[idx, trial, :, state-1], 
                                            color=color)
                            ax.set_xlim([ds["time"].values[0], ds["time"].values[-1]])
                            ax.set_ylim([0, 1])
                            ax.set_ylabel(f"Prob. st{state} tr{trial}")
                    ax.set_xlabel("Time (s)")
                    fig.savefig(figures_dir + f'tcourses-{n_states}states-subj{subj}-IC{IC}.png', dpi=300)
                    plt.close(fig)
            except:
                print(f"IC{IC} of subject {subj} does not exist")